### Making connection to database

In [1]:
from mysql_connection import MySqlManip
from neo4j_connection import Neo4jManip
import pandas as pd

conn = MySqlManip(
    user='root', host='10.60.5.99', port=3306, password='Boway@123', database='IOT'
)

# conn.create_params_table()

### Reading historian data

In [2]:
data = conn.load_data_to_dataframe('device_4_bubble_cleaned_data_bayes')
data.dropna()
selected_labels = ["cool_strength_total_mean_3min", "temp_3m_jiaozhu", "label"]
data = data[selected_labels]

length = len(data)
cool_strength_upper_threshold = sorted(data["cool_strength_total_mean_3min"].tolist())[int(length * 4 / 5)]
cool_strength_lower_threshold = sorted(data["cool_strength_total_mean_3min"].tolist())[int(length * 1 / 5)]
temp_upper_threshold = sorted(data["temp_3m_jiaozhu"].tolist())[int(length * 5 / 6)]
temp_lower_threshold = sorted(data["temp_3m_jiaozhu"].tolist())[int(length * 1 / 5)]

data["cool_strength_total_mean_3min"] = pd.cut(
    data["cool_strength_total_mean_3min"],
    [-float('inf'), cool_strength_lower_threshold, cool_strength_upper_threshold, float('inf')],
    labels=["IncompleteCooled","WellCooled","OverCooled"])

data["temp_3m_jiaozhu"] = pd.cut(
    data["temp_3m_jiaozhu"],
    [-float('inf'), temp_lower_threshold, temp_upper_threshold, float('inf')],
    labels=["UnderHeated","WellHeated","OverHeated"])

data["label"] = pd.cut(
    data["label"],
    2,
    labels=["FineBatch","BadBatch"])

### Creating BN model

In [3]:
from models.bayes_net import BNCreator
bn = BNCreator(["cool_strength_total_mean_3min", "temp_3m_jiaozhu"], 'label')

Fitting model with data (initialize model)

In [4]:
bn.model.fit(data)
bn.model.get_cpds()[0].state_names

{'cool_strength_total_mean_3min': ['IncompleteCooled',
  'OverCooled',
  'WellCooled']}

In [5]:
print(bn.model.get_cpds()[0])

+-------------------------------------------------+----------+
| cool_strength_total_mean_3min(IncompleteCooled) | 0.200086 |
+-------------------------------------------------+----------+
| cool_strength_total_mean_3min(OverCooled)       | 0.19984  |
+-------------------------------------------------+----------+
| cool_strength_total_mean_3min(WellCooled)       | 0.600074 |
+-------------------------------------------------+----------+


Making changes towards cpd (prior probability)

In [6]:
bn.update_cpd(
    "cool_strength_total_mean_3min",
    {
        "IncompleteCooled": 0.08,
        "OverCooled": 0.13,
        "WellCooled": 0.79
    }
)

In [7]:
print(bn.model.get_cpds()[0])

+-------------------------------------------------+------+
| cool_strength_total_mean_3min(IncompleteCooled) | 0.08 |
+-------------------------------------------------+------+
| cool_strength_total_mean_3min(OverCooled)       | 0.13 |
+-------------------------------------------------+------+
| cool_strength_total_mean_3min(WellCooled)       | 0.79 |
+-------------------------------------------------+------+


In [10]:
print(bn.model.get_cpds()[1])

+-------------------------------+-----+-------------------------------------------+
| cool_strength_total_mean_3min | ... | cool_strength_total_mean_3min(WellCooled) |
+-------------------------------+-----+-------------------------------------------+
| temp_3m_jiaozhu               | ... | temp_3m_jiaozhu(WellHeated)               |
+-------------------------------+-----+-------------------------------------------+
| label(BadBatch)               | ... | 0.13519985264321238                       |
+-------------------------------+-----+-------------------------------------------+
| label(FineBatch)              | ... | 0.8648001473567877                        |
+-------------------------------+-----+-------------------------------------------+


In [11]:
print(bn.model.get_cpds()[2])

+------------------------------+----------+
| temp_3m_jiaozhu(OverHeated)  | 0.165114 |
+------------------------------+----------+
| temp_3m_jiaozhu(UnderHeated) | 0.300993 |
+------------------------------+----------+
| temp_3m_jiaozhu(WellHeated)  | 0.533893 |
+------------------------------+----------+


### Inference

In [8]:
sample_data = data[["cool_strength_total_mean_3min", "temp_3m_jiaozhu"]].loc[[10300]]

demon_data = pd.DataFrame( sample_data)
demon_data.columns = ["冷却强度", "浇铸温度"]
demon_data

,冷却强度,浇铸温度
10300,WellCooled,OverHeated


In [9]:
res = bn.model.predict_probability(sample_data)

res.columns = ["有气泡批次", "无气泡批次"]
res

,有气泡批次,无气泡批次
10300,0.048319,0.951681
